In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import re
from fancyimpute import IterativeImputer
import locale
from sqlalchemy import create_engine
import warnings

# Ignorar warnings
warnings.filterwarnings("ignore")

### ETL (Extraction, Transformation, Load)

En primer lugar, procedemos a cargar el dataset

In [2]:
df_hechos = pd.read_excel(r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI2\DataLake\Obligatorio\homicidios.xlsx", sheet_name = "HECHOS")
df_victimas = pd.read_excel(r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI2\DataLake\Obligatorio\homicidios.xlsx", sheet_name = "VICTIMAS")

Procedemos a ver las dimensiones del dataframe

In [3]:
df_hechos.shape

(696, 21)

Visualizamos

In [4]:
df_hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Procedemos a chequear nulos

In [5]:
df_hechos.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

In [6]:
porcentaje_nulos = {}
for column in df_hechos.columns:
    porcentaje = df_hechos[column].isnull().sum() / df_hechos.shape[0]*100
    porcentaje_nulos[column] = round(porcentaje,2)
for column, percentage in porcentaje_nulos.items():
    print(f"{column:} {percentage}%")

ID 0.0%
N_VICTIMAS 0.0%
FECHA 0.0%
AAAA 0.0%
MM 0.0%
DD 0.0%
HORA 0.0%
HH 0.0%
LUGAR_DEL_HECHO 0.0%
TIPO_DE_CALLE 0.0%
Calle 0.14%
Altura 81.47%
Cruce 24.57%
Dirección Normalizada 1.15%
COMUNA 0.0%
XY (CABA) 0.0%
pos x 0.0%
pos y 0.0%
PARTICIPANTES 0.0%
VICTIMA 0.0%
ACUSADO 0.0%


In [7]:
col_drop = {}
for column, percentage in porcentaje_nulos.items():
        if percentage > 50:
            col_drop[column] = porcentaje_nulos[column]

print("Las columnas que se deben eliminar debido a su gran cantidad de valores nulos son: ")
for column in col_drop.keys():
      print(column)

Las columnas que se deben eliminar debido a su gran cantidad de valores nulos son: 
Altura


Podemos ver que la gran mayoria de datos de esta columna son nulos. Así que procederemos a eliminarla


In [8]:
to_delete = []
for column in col_drop.keys():
    to_delete.append(column)
df_hechos.drop(columns=(to_delete),inplace = True)
df_hechos.shape

(696, 20)

In [9]:
df_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


La otra columna que posee una gran cantidad de valores nulos es "Cruce". Vamos a analizarla para ver la viabilidad de una imputacion de datos

In [10]:
df_hechos["Cruce"].value_counts()

Cruce
PAZ, GRAL. AV.              14
ESCALADA AV.                 9
RIVADAVIA AV.                8
INDEPENDENCIA AV.            7
BALBIN, RICARDO, DR. AV.     6
                            ..
LA RIOJA                     1
TRAFUL                       1
HUMAITA                      1
MANZANARES                   1
HUGO, VICTOR                 1
Name: count, Length: 317, dtype: int64

Veremos los valores que son nulos en estas tres columnas. Ya que de ser asi, no poseen informacion util para nosotros

In [11]:

filas_nulas = df_hechos[df_hechos[["Calle", "Cruce", "Dirección Normalizada"]].isnull().any(axis=1)]
print(filas_nulas)
print(len(filas_nulas))



            ID  N_VICTIMAS      FECHA  AAAA  MM  DD      HORA  HH  \
2    2016-0003           1 2016-01-03  2016   1   3  07:00:00   7   
9    2016-0013           1 2016-02-10  2016   2  10  11:30:00  11   
14   2016-0020           1 2016-02-17  2016   2  17  16:00:00  16   
33   2016-0047           1 2016-04-15  2016   4  15  05:30:00   5   
35   2016-0049           1 2016-04-17  2016   4  17  00:00:00   0   
..         ...         ...        ...   ...  ..  ..       ...  ..   
680  2021-0082           1 2021-11-06  2021  11   6  04:39:00   4   
685  2021-0087           1 2021-11-27  2021  11  27  11:22:00  11   
687  2021-0089           1 2021-12-02  2021  12   2  01:10:00   1   
694  2021-0096           1 2021-12-15  2021  12  15  10:30:00  10   
695  2021-0097           1 2021-11-18  2021  11  18  06:10:00   6   

                                       LUGAR_DEL_HECHO TIPO_DE_CALLE  \
2                                   AV ENTRE RIOS 2034       AVENIDA   
9                          

Son varias columnas, por lo tanto no se eliminarán ya que pueden llevar a obtener conclusiones relevantes


Ahora, se normalizarán los nombres de las columnas para que su esteticidad sea mas profesional


In [12]:
df_hechos.columns = df_hechos.columns.str.title()
df_hechos.head(5)

,Id,N_Victimas,Fecha,Aaaa,Mm,Dd,Hora,Hh,Lugar_Del_Hecho,Tipo_De_Calle,Calle,Cruce,Dirección Normalizada,Comuna,Xy (Caba),Pos X,Pos Y,Participantes,Victima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Procederemos a ver duplicados de la columna Id

In [13]:
df_hechos["Id"].value_counts() # No hay valores duplicados

Id
2016-0001    1
2019-0050    1
2019-0042    1
2019-0043    1
2019-0044    1
            ..
2017-0111    1
2017-0112    1
2017-0113    1
2017-0114    1
2021-0097    1
Name: count, Length: 696, dtype: int64

In [14]:
id_dup = df_hechos[df_hechos.duplicated(subset=["Id"])] # Rectificamos con otra alternativa
print(len(id_dup))

0


In [15]:
# Procederemos a analizar la columna Hh
df_hechos["Hh"].value_counts()
# Segun el diccionario de datos esta columna se refiere a "Franja horaria entera". Se refiere a la hora en la cual el incidente tuvo lugar
# Por lo tanto deberia tener solamente valores numericos. Sin embargo, se identifica un valor "SD". El cual se refiere a "Sin Dato"
# Por lo tanto debemos analizar como trabajar este dato.

Hh
7     41
6     40
9     36
5     35
14    33
12    32
8     31
18    31
10    31
16    30
19    30
17    30
22    30
21    29
23    28
11    27
3     26
20    26
15    25
1     24
4     23
0     23
2     17
13    17
SD     1
Name: count, dtype: int64

In [16]:
# Transformaremos SD a valor nulo o NaN
df_hechos.replace(['SD'], np.nan, inplace=True)
df_hechos = df_hechos.infer_objects(copy=False) # para evitar warnings

Seguimos trabajando en esta columna. Siguiendo el contexto esta columna deberia ser numerica. Pero, debido a que habia un dato "SD" pueda que esto no sea asi. Asi que, se procederá a chequearlas.


In [17]:
print(df_hechos["Hh"].dtype)

float64


Se procedera a cambiarlos a valor entero

In [18]:
df_hechos["Hh"] = df_hechos["Hh"].astype("Int64") 

Ahora, se trabajará la columna de las posiciones "pos x" y "pos y" vamos a limpiarla.


In [19]:
# Hacemos una funcion para encontrar posiciones invalidas, covertirlas a nulo. O si son validas convrtirlas a float
def validar_y_convertir(pos):
    if pos in ['.', None] or not re.match(r'^[0-9.,-]+$', str(pos)):
        return np.nan
    return float(str(pos).replace(',', '.'))

# Aplicar la función para validar y convertir las posiciones
df_hechos['Pos X'] = df_hechos['Pos X'].apply(validar_y_convertir)
df_hechos['Pos Y'] = df_hechos['Pos Y'].apply(validar_y_convertir)

Dirigiremos nuestra atencion a la columna XY (CABA). Debemos desanidarla ya que allí se encuentran coordenadas (latitud y longitd). Vamos a separarlas

In [20]:
# Hacemos una funcion para extraer las coordenadas con su signo
def extraer_coordenadas_signo(texto):
    # Expresión regular que también admite signos negativos
    coordenadas = re.findall(r'-?\d+\.\d+', texto)
    if len(coordenadas) == 2:
        return float(coordenadas[0]), float(coordenadas[1])
    else:
        return None, None

In [21]:
df_hechos['Coordenada_x'],df_hechos['Coordenada_y'] = zip(*df_hechos['Xy (Caba)'].map(extraer_coordenadas_signo)) # Aplicamos la funcion

In [22]:
df_hechos[["Coordenada_x", "Coordenada_y"]] = df_hechos[["Coordenada_x", "Coordenada_y"]].dropna() # Eliminamos nulos


In [23]:
df_hechos.head(2)

,Id,N_Victimas,Fecha,Aaaa,Mm,Dd,Hora,Hh,Lugar_Del_Hecho,Tipo_De_Calle,...,Dirección Normalizada,Comuna,Xy (Caba),Pos X,Pos Y,Participantes,Victima,Acusado,Coordenada_x,Coordenada_y
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420


Procederemos con la columna de "Día". Ya que estan con digitos vamos a cambiarla por el nombre del día.

In [24]:
df_hechos['Nombre_Día'] = df_hechos['Fecha'].dt.day_name(locale='es')




In [25]:
df_hechos.head(5)

,Id,N_Victimas,Fecha,Aaaa,Mm,Dd,Hora,Hh,Lugar_Del_Hecho,Tipo_De_Calle,...,Comuna,Xy (Caba),Pos X,Pos Y,Participantes,Victima,Acusado,Coordenada_x,Coordenada_y,Nombre_Día
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,Viernes
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,Sábado
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878,Domingo
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,NaN,99840.652248,94269.165344,Domingo
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,106980.328279,100752.169158,Jueves


Ahora haremos lo mismo con los meses del año.

In [26]:
meses_año = {
    1: 'Enero',
    2: 'Febrero',
    3: 'Marzo',
    4: 'Abril',
    5: 'Mayo',
    6: 'Junio',
    7: 'Julio',
    8: 'Agosto',
    9: 'Septiembre',
    10: 'Octubre',
    11: 'Noviembre',
    12: 'Diciembre'
}
# Mapeamos para reemplazar los valores
df_hechos["Mm"] = df_hechos["Mm"].map(meses_año)

In [27]:
# Eliminamos la columna Xy (Caba)
df_hechos.drop(columns=['Xy (Caba)'], inplace = True)

Ahora procedemos a normalizar el nombre de las columnas.

In [28]:
df_hechos.columns = df_hechos.columns.str.title()


In [29]:
# Cambiamos el nombre de algunas columnas
df_hechos.rename(columns={'Aaaa': "Año", "Mm":"Mes", "Dd":"Día", "Hh":"Franja Horaria Entera" }, inplace = True)

In [30]:
nuevo_orden = ["Id","N_Victimas","Fecha", "Año", "Mes", "Día","Nombre_Día", "Hora", "Franja Horaria Entera", "Lugar_Del_Hecho", "Tipo_De_Calle", "Calle", "Cruce", "Dirección Normalizada", "Comuna", "Pos X", "Pos Y",  "Coordenada_X", "Coordenada_Y", "Participantes", "Victima", "Acusado"]

df_hechos = df_hechos[nuevo_orden]

Vamos a verificar el tipo de dato de las columnas. Y a cambiar cuando sea necesario.

In [31]:
print(df_hechos.dtypes)

Id                               object
N_Victimas                        int64
Fecha                    datetime64[ns]
Año                               int64
Mes                              object
Día                               int64
Nombre_Día                       object
Hora                             object
Franja Horaria Entera             Int64
Lugar_Del_Hecho                  object
Tipo_De_Calle                    object
Calle                            object
Cruce                            object
Dirección Normalizada            object
Comuna                            int64
Pos X                           float64
Pos Y                           float64
Coordenada_X                    float64
Coordenada_Y                    float64
Participantes                    object
Victima                          object
Acusado                          object
dtype: object


In [32]:
# Convertimos a tipo datetime y luego extraer Año, Mes y Día
df_hechos['Fecha'] = pd.to_datetime(df_hechos['Fecha'], errors='coerce')
df_hechos['Año'] = pd.to_datetime(df_hechos['Año'], format='%Y')


# Convertimos Hora a tipo de tiempo
df_hechos['Hora'] = pd.to_datetime(df_hechos['Hora'], format='%H:%M:%S', errors='coerce').dt.time

# Las columnas de texto se pasan a str
df_hechos['Lugar_Del_Hecho'] = df_hechos['Lugar_Del_Hecho'].astype(str)
df_hechos['Tipo_De_Calle'] = df_hechos['Tipo_De_Calle'].astype(str)
df_hechos['Calle'] = df_hechos['Calle'].astype(str)
df_hechos['Cruce'] = df_hechos['Cruce'].astype(str)
df_hechos['Dirección Normalizada'] = df_hechos['Dirección Normalizada'].astype(str)
df_hechos['Pos X'] = pd.to_numeric(df_hechos['Pos X'], errors='coerce')  
df_hechos['Pos Y'] = pd.to_numeric(df_hechos['Pos Y'], errors='coerce')  
df_hechos['Participantes'] = df_hechos['Participantes'].astype(str)
df_hechos['Victima'] = df_hechos['Victima'].astype(str)
df_hechos['Acusado'] = df_hechos['Acusado'].astype(str)



In [33]:
df_hechos.head(2)

,Id,N_Victimas,Fecha,Año,Mes,Día,Nombre_Día,Hora,Franja Horaria Entera,Lugar_Del_Hecho,...,Cruce,Dirección Normalizada,Comuna,Pos X,Pos Y,Coordenada_X,Coordenada_Y,Participantes,Victima,Acusado
0,2016-0001,1,2016-01-01,2016-01-01,Enero,1,Viernes,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,...,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.475340,-34.687570,98896.782384,93532.434378,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016-01-01,Enero,2,Sábado,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,...,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.508775,-34.669777,95832.055711,95505.416420,AUTO-PASAJEROS,AUTO,PASAJEROS


Vamos a corregir la columna "Año"

In [34]:
df_hechos['Fecha'] = pd.to_datetime(df_hechos['Fecha'])

# Extraemos solo el año de la columna 'Fecha' y actualizamos la columna 'Año'
df_hechos['Año'] = df_hechos['Fecha'].dt.year

# Mostrar el DataFrame resultante
df_hechos.head(3)

,Id,N_Victimas,Fecha,Año,Mes,Día,Nombre_Día,Hora,Franja Horaria Entera,Lugar_Del_Hecho,...,Cruce,Dirección Normalizada,Comuna,Pos X,Pos Y,Coordenada_X,Coordenada_Y,Participantes,Victima,Acusado
0,2016-0001,1,2016-01-01,2016,Enero,1,Viernes,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,...,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.475340,-34.687570,98896.782384,93532.434378,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,Enero,2,Sábado,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,...,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.508775,-34.669777,95832.055711,95505.416420,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,Enero,3,Domingo,07:00:00,7,AV ENTRE RIOS 2034,...,nan,ENTRE RIOS AV. 2034,1,-58.390403,-34.631894,106684.290900,99706.576878,MOTO-AUTO,MOTO,AUTO


In [35]:
#df_hechos.to_csv("hechos.csv", sep = ";", index = False) # Exportamos a csv

### Ahora procederemos a ver el dataset de victimas.


In [36]:
df_victimas.head(5)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


Primero veremos datos nulos.

In [37]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [38]:
df_victimas.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

No hay nulos. Asi que procederemos a ver duplicados en la columna de Id.

In [39]:
df_victimas["ID_hecho"].value_counts()

ID_hecho
2017-0035    3
2017-0050    2
2016-0041    2
2018-0015    2
2020-0063    2
            ..
2017-0115    1
2017-0116    1
2017-0118    1
2017-0120    1
2021-0096    1
Name: count, Length: 696, dtype: int64

Si hay duplicados. Asi que los vamos a examinar.

In [40]:
duplicados = df_victimas[df_victimas.duplicated(subset='ID_hecho', keep=False)]
print(duplicados)


      ID_hecho      FECHA  AAAA  MM  DD                   ROL VICTIMA  \
29   2016-0041 2016-03-29  2016   3  29             CONDUCTOR    MOTO   
30   2016-0041 2016-03-29  2016   3  29  PASAJERO_ACOMPAÑANTE    MOTO   
98   2016-0126 2016-09-18  2016   9  18             CONDUCTOR    AUTO   
99   2016-0126 2016-09-18  2016   9  18  PASAJERO_ACOMPAÑANTE    AUTO   
163  2017-0026 2017-02-26  2017   2  26  PASAJERO_ACOMPAÑANTE    AUTO   
164  2017-0026 2017-02-26  2017   2  26             CONDUCTOR    AUTO   
173  2017-0035 2017-03-23  2017   3  23             CONDUCTOR    AUTO   
174  2017-0035 2017-03-23  2017   3  23  PASAJERO_ACOMPAÑANTE    AUTO   
175  2017-0035 2017-03-23  2017   3  23  PASAJERO_ACOMPAÑANTE    AUTO   
176  2017-0036 2017-03-29  2017   3  29  PASAJERO_ACOMPAÑANTE    MOTO   
177  2017-0036 2017-03-29  2017   3  29             CONDUCTOR    MOTO   
186  2017-0050 2017-04-28  2017   4  28             CONDUCTOR    MOTO   
187  2017-0050 2017-04-28  2017   4  28  PASAJERO_A

Vamos a conservar los duplicados ya que tienen distinta informacion en cuanto al rol.

Siguiente, examinaremos el tipo de dato de las columnas.

In [41]:
column_types = pd.DataFrame(df_victimas.dtypes, columns=['Tipo de Dato'])
print(column_types)


                       Tipo de Dato
ID_hecho                     object
FECHA                datetime64[ns]
AAAA                          int64
MM                            int64
DD                            int64
ROL                          object
VICTIMA                      object
SEXO                         object
EDAD                         object
FECHA_FALLECIMIENTO          object


In [42]:
df_victimas["ROL"] = df_victimas["ROL"].astype(str) 
df_victimas["VICTIMA"] = df_victimas["VICTIMA"].astype(str) 
df_victimas["SEXO"] = df_victimas["SEXO"].astype(str)


In [43]:
df_victimas["FECHA_FALLECIMIENTO"] = pd.to_datetime(df_victimas["FECHA_FALLECIMIENTO"], format='%Y-%m-%d', errors='coerce')
df_victimas ["EDAD"] = pd.to_numeric(df_victimas["EDAD"], errors="coerce")
df_victimas ["EDAD"] = df_victimas["EDAD"].astype("Int64")

Ahora chequeamos la columna de edad.

In [44]:
df_victimas["EDAD"].unique() # Podemos ver que hay un valor nulo

<IntegerArray>
[  19,   70,   30,   18,   29,   22,   16,   59,   65,   34,   41,   50,   38,
   21,   52,   36,   20,   54, <NA>,   56,   24,   78,   79,   26,   57,   37,
   58,   23,   60,   42,   53,   51,   40,   87,   76,   75,   35,   80,   43,
   45,   67,   27,   55,   49,   81,   25,   33,   46,   83,   39,   28,    7,
   48,    4,   82,   32,   17,   47,   61,   10,   95,   73,   84,   66,   85,
    1,   15,   13,   77,   44,   31,   62,   74,   71,   11,   86,   69,   72,
   12,   63,   92,   68,   91,   64,    5,   88]
Length: 86, dtype: Int64

In [45]:
df_victimas["EDAD"].isnull().sum()


53

In [46]:
edad_nulos = {}
for column in df_victimas.columns:
    porcentaje = (df_victimas[column].isnull().sum() / df_victimas.shape[0])*100
    edad_nulos[column] = round(porcentaje,2)
print(edad_nulos)

{'ID_hecho': 0.0, 'FECHA': 0.0, 'AAAA': 0.0, 'MM': 0.0, 'DD': 0.0, 'ROL': 0.0, 'VICTIMA': 0.0, 'SEXO': 0.0, 'EDAD': 7.39, 'FECHA_FALLECIMIENTO': 9.62}


A pesar de que la cantidad de nulos es baja, la considero una variable importante. Asi que lleno esos valores nulos con la tecnica de Imputacion Multiple, usando la libreria FancyImpute

In [47]:
# Primero creo una copia del dataframe original

df_imputacion = df_victimas.copy()
imputer = IterativeImputer() # Instancio el modelo
df_imputacion[["EDAD"]] = imputer.fit_transform(df_imputacion[["EDAD"]]) # Entreno el modelo
df_victimas["EDAD"] = df_imputacion["EDAD"]


In [48]:
df_victimas["EDAD"].isnull().sum() #Chequeamos

0

Ahora haremos lo mismo con la columna de Fecha de Nacimiento.

In [49]:
# Creamos una columna numérica a partir de la fecha y aplicamos la imputación múltiple
df_victimas['FECHA_FALLECIMIENTO_num'] = df_victimas['FECHA_FALLECIMIENTO'].astype(np.int64) // 10**9
imputer = IterativeImputer(max_iter = 10, random_state = 0)

# Entrenamos el modelo y realizamos la imputación
df_victimas['FECHA_FALLECIMIENTO_num'] = imputer.fit_transform(df_victimas[['FECHA_FALLECIMIENTO_num']])

# Verificamos los valores imputados y nos aseguramos que estén dentro de un rango razonable
min_timestamp = pd.to_datetime('1900-01-01').timestamp()
max_timestamp = pd.to_datetime('2100-12-31').timestamp()

# Limitamos los valores a un rango válido
df_victimas['FECHA_FALLECIMIENTO_num'] = df_victimas['FECHA_FALLECIMIENTO_num'].clip(lower=min_timestamp, upper=max_timestamp)

# Convertimos de nuevo a tipo datetime
df_victimas['FECHA_FALLECIMIENTO'] = pd.to_datetime(df_victimas['FECHA_FALLECIMIENTO_num'], unit='s')

# Eliminamos la columna numérica auxiliar
df_victimas.drop(columns=['FECHA_FALLECIMIENTO_num'], inplace=True)


In [50]:
df_victimas['FECHA_FALLECIMIENTO'].isnull().sum() # Ya no hay nulos

0

In [51]:
df_victimas.head(5)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,1900-01-01
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01


La columna "EDAD" tiene decimales. Vamos a limpiarla.

In [52]:
df_victimas['EDAD'] = df_victimas['EDAD'].astype(int)


In [53]:
df_victimas.head(5)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,1900-01-01
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01


Ahora vamos a crear un rango etario para mejorar el analisis de la variable "EDAD"

In [54]:
rangos = [-float('inf'), 20, 30, 40, 50, 60, float('inf')]
labels = ['Menos de 20', 'Entre 20 y 30', 'Entre 30 y 40', 'Entre 40 y 50', 'Entre 50 y 60', 'Mayores de 60']

# Asignar los rangos etarios
df_victimas['Rango_Etario'] = pd.cut(df_victimas['EDAD'], bins=rangos, labels=labels, right=False)

# Mostrar el DataFrame con el rango etario
df_victimas.head(5)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Rango_Etario
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,Menos de 20
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,Mayores de 60
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,Entre 30 y 40
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,1900-01-01,Menos de 20
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01,Entre 20 y 30


Ahora procedemos a analizar la columna mes. Vamos a reemplazar el digito por el nombre de los meses.

In [55]:
# Creamos un diccionario para mapear luego
meses = {
    1: 'Enero',
    2: 'Febrero',
    3: 'Marzo',
    4: 'Abril',
    5: 'Mayo',
    6: 'Junio',
    7: 'Julio',
    8: 'Agosto',
    9: 'Septiembre',
    10: 'Octubre',
    11: 'Noviembre',
    12: 'Diciembre'
}

df_victimas["MM"] = df_victimas["MM"].map(meses) 

Se hará lo mismo con la columna Día. Solo que no se reemplazaran los datos sino que se agregara una nueva columna.

In [56]:
df_victimas['Nombre_Día'] = df_victimas['FECHA'].dt.day_name(locale = 'es')

In [57]:
df_victimas.head(5)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Rango_Etario,Nombre_Día
0,2016-0001,2016-01-01,2016,Enero,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,Menos de 20,Viernes
1,2016-0002,2016-01-02,2016,Enero,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,Mayores de 60,Sábado
2,2016-0003,2016-01-03,2016,Enero,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,Entre 30 y 40,Domingo
3,2016-0004,2016-01-10,2016,Enero,10,CONDUCTOR,MOTO,MASCULINO,18,1900-01-01,Menos de 20,Domingo
4,2016-0005,2016-01-21,2016,Enero,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01,Entre 20 y 30,Jueves


Ahora procederemos a cambiar el nombre de algunas columnas

In [58]:
df_victimas.rename(columns={"AAAA":"Año", "MM":"Mes", "DD":"Día", "SEXO":"GENERO"}, inplace = True)

In [59]:
df_victimas.columns = df_victimas.columns.str.title() 

In [60]:
df_victimas.columns

Index(['Id_Hecho', 'Fecha', 'Año', 'Mes', 'Día', 'Rol', 'Victima', 'Genero',
       'Edad', 'Fecha_Fallecimiento', 'Rango_Etario', 'Nombre_Día'],
      dtype='object')

Procederé a cambiar el orden de las columnas para mejor visualización.

In [61]:
nuevo_orden = ['Id_Hecho', 'Fecha', 'Año', 'Mes', 'Día','Nombre_Día', 'Rol', 'Victima', 'Genero', 'Edad',  'Rango_Etario',  'Fecha_Fallecimiento']
df_victimas = df_victimas[nuevo_orden]

In [62]:
df_victimas.head(2)

,Id_Hecho,Fecha,Año,Mes,Día,Nombre_Día,Rol,Victima,Genero,Edad,Rango_Etario,Fecha_Fallecimiento
0,2016-0001,2016-01-01,2016,Enero,1,Viernes,CONDUCTOR,MOTO,MASCULINO,19,Menos de 20,2016-01-01
1,2016-0002,2016-01-02,2016,Enero,2,Sábado,CONDUCTOR,AUTO,MASCULINO,70,Mayores de 60,2016-01-02


In [63]:
df_victimas.dtypes

Id_Hecho                       object
Fecha                  datetime64[ns]
Año                             int64
Mes                            object
Día                             int64
Nombre_Día                     object
Rol                            object
Victima                        object
Genero                         object
Edad                            int32
Rango_Etario                 category
Fecha_Fallecimiento    datetime64[ns]
dtype: object

Procederemos a evaluar las columnas, ya sea, aisladamente o en conjunto.

Procedemos a exportar a archivo csv

In [64]:
df_victimas.to_csv("victimas.csv", sep = ";", index = False)

Vamos a unir los dos datasets para tenerlos disponibles en caso de ser requeridos para futuros analisis.

In [65]:
df_merged = pd.merge(df_victimas, df_hechos, left_on="Id_Hecho", right_on="Id", how="inner")


In [66]:
df_merged.columns

Index(['Id_Hecho', 'Fecha_x', 'Año_x', 'Mes_x', 'Día_x', 'Nombre_Día_x', 'Rol',
       'Victima_x', 'Genero', 'Edad', 'Rango_Etario', 'Fecha_Fallecimiento',
       'Id', 'N_Victimas', 'Fecha_y', 'Año_y', 'Mes_y', 'Día_y',
       'Nombre_Día_y', 'Hora', 'Franja Horaria Entera', 'Lugar_Del_Hecho',
       'Tipo_De_Calle', 'Calle', 'Cruce', 'Dirección Normalizada', 'Comuna',
       'Pos X', 'Pos Y', 'Coordenada_X', 'Coordenada_Y', 'Participantes',
       'Victima_y', 'Acusado'],
      dtype='object')

Vamos a ver si las columnas son sufijo _x son iguales a aquellas cuyo sufijo es _y

In [67]:

columnas_x = [col for col in df_merged.columns if col.endswith('_x')]
columnas_y = [col for col in df_merged.columns if col.endswith('_y')]

if len(columnas_x) == len(columnas_y):
        for col_x, col_y in zip(columnas_x, columnas_y):
            iguales = df_merged[col_x].equals(df_merged[col_y])  
            print(f"¿Las columnas {col_x} y {col_y} son iguales?: {iguales}")
else:
    print("El número de columnas con _x y _y no coincide.")


¿Las columnas Fecha_x y Fecha_y son iguales?: False
¿Las columnas Año_x y Año_y son iguales?: False
¿Las columnas Mes_x y Mes_y son iguales?: True
¿Las columnas Día_x y Día_y son iguales?: False
¿Las columnas Nombre_Día_x y Nombre_Día_y son iguales?: False
¿Las columnas Victima_x y Victima_y son iguales?: False


Podemos ver que las columnas Mes. Son iguales asi que procederemos a eliminar una de estas.

In [68]:
df_merged.drop(columns=["Mes_y"], inplace = True)
df_merged.rename(columns={"Mes_x":"Mes"}, inplace = True)

Ahora procederemos a cambiar el sufijo por el nombre del dataset de procedencia.

In [69]:

df_merged.rename(columns=lambda col: col.replace('_x', '_Victima') if col.endswith('_x') else col.replace('_y', '_Hechos'), inplace=True)

# Verificar los cambios en las columnas
print(df_merged.columns)


Index(['Id_Hecho', 'Fecha_Victima', 'Año_Victima', 'Mes', 'Día_Victima',
       'Nombre_Día_Victima', 'Rol', 'Victima_Victima', 'Genero', 'Edad',
       'Rango_Etario', 'Fecha_Fallecimiento', 'Id', 'N_Victimas',
       'Fecha_Hechos', 'Año_Hechos', 'Día_Hechos', 'Nombre_Día_Hechos', 'Hora',
       'Franja Horaria Entera', 'Lugar_Del_Hecho', 'Tipo_De_Calle', 'Calle',
       'Cruce', 'Dirección Normalizada', 'Comuna', 'Pos X', 'Pos Y',
       'Coordenada_X', 'Coordenada_Y', 'Participantes', 'Victima_Hechos',
       'Acusado'],
      dtype='object')


Procedemos a exportarlo.

In [70]:
df_merged.to_csv("Merged.csv", sep = ";" ,index = False)

In [71]:
info = pd.read_csv(r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI2\DataLake\Opcional\data.csv", sep = ";")


In [73]:
# Configuración de conexión
username = info.iloc[0,1]
password = info.iloc[1,1]
host = 'localhost'
port = '3306'
database = info.iloc[2,1]

# Crear la URL de conexión
url = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(url)

try:
    connection = engine.connect()
    print("Conexión con la base de datos MySQL exitosa.")
except Exception as e:
    print(f"Error al conectar con la base de datos MySQL: {e}")

df_hechos.to_sql("Hechos_1", engine, if_exists = "replace", index = False)
df_victimas.to_sql("Victimas_1", engine, if_exists = "replace", index = False)
df_merged.to_sql("Siniestros", engine, if_exists = "replace", index = False)

# Cerrar la conexión
engine.dispose()

Conexión con la base de datos MySQL exitosa.
